In [27]:
#import modules
import pandas as pd

#list all csv files only
csv_files = ['Data/tns_search (14).csv', 'Data/tns_search (5).csv', 'Data/tns_search (6).csv', 'Data/tns_search (7).csv', 'Data/tns_search (8).csv', 'Data/tns_search (9).csv']


In [28]:
df_append = pd.DataFrame()

#append all files together
for file in csv_files:
    df_temp = pd.read_csv(file)
    df_append = pd.concat([df_append, df_temp], ignore_index=True)


In [29]:
df_append.to_csv("Data/ZTF_info.csv")